In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import gensim
import re
import string
import nltk
from gensim import corpora, models, similarities, matutils
from sklearn.metrics.pairwise import cosine_similarity

# 1. Data

In [3]:
df.head()

,business_id,cool,funny,stars,useful,text_clean,text_noun
0,--9e1ONYQuAa-CB_Rrw7Tw,0.631389,0.471562,4.087113,0.942405,what can i say wowzers probably one of the be...,wowzers steak houses service dinner seafood to...
1,0AQnRQw34IQW9-1gJkYnMA,0.307479,0.323176,2.943675,0.598338,me and my best friend stayed at the monte car...,friend monte carlo diablo s hotel didn feel ve...
2,0NmTwqYEQiKErDv4a55obg,0.801971,0.573477,4.120968,1.252688,stars goes to the truffled mushroom with pole...,stars mushroom stars basil spaghetti ambience ...
3,2weQS-RnoOBhb1KsHKyoSQ,0.767622,0.734573,3.637749,1.209140,over one hour wait for sunday brunch not a hu...,hour wait brunch selection breakfast items des...
4,4GXII-GU7S0ZyU6ElkhscQ,0.496835,0.787975,2.352848,1.131329,yeah this place is a friggin joke i live hear...,yeah place joke i hear pool sulfar pool deck f...


# 2. Text preprocessing

In [ ]:
def preprocess(df):
    alphabet = lambda x: re.sub(r"""[^a-z]+""", ' ', x)  # remove non-alphabets
    punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower()) # remove punctuation and lower the case
    enter = lambda x: x.replace('\n','')  # remove \n

    df['text_clean'] = df.text.map(punc_lower).map(enter).map(alphabet)

    # Only keep nouns
    noun=[]
    for item in df.text_clean:     
        tokens = nltk.word_tokenize(item)
        tags = nltk.pos_tag(tokens)
        nouns = [word for word,pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]
        noun.append(nouns)

    df['text_noun'] = noun
    for i in range(len(df.text_noun)):     
        df.text_noun[i] = ' '.join(df.text_noun[i])

    # Remove some common nouns
    mapping = { 'food':'', 'place':'', 'time':'', 'vegas':'', 'people':'','strip':'','service':'','night':''}

    for i in range(len(df.text_noun)):
        for k, v in mapping.items():
            df.text_noun[i] = df.text_noun[i].replace(k, v)
    return df

df = preprocess(df)

In [9]:
df.head()

,business_id,cool,funny,stars,useful,text_clean,text_noun
0,--9e1ONYQuAa-CB_Rrw7Tw,0.631389,0.471562,4.087113,0.942405,what can i say wowzers probably one of the be...,wowzers steak houses dinner sea tower wedge w...
1,0AQnRQw34IQW9-1gJkYnMA,0.307479,0.323176,2.943675,0.598338,me and my best friend stayed at the monte car...,friend monte carlo diablo s hotel didn feel d...
2,0NmTwqYEQiKErDv4a55obg,0.801971,0.573477,4.120968,1.252688,stars goes to the truffled mushroom with pole...,stars mushroom stars basil spaghetti ambience ...
3,2weQS-RnoOBhb1KsHKyoSQ,0.767622,0.734573,3.637749,1.209140,over one hour wait for sunday brunch not a hu...,hour wait brunch selection breakfast items des...
4,4GXII-GU7S0ZyU6ElkhscQ,0.496835,0.787975,2.352848,1.131329,yeah this place is a friggin joke i live hear...,yeah joke i hear pool sulfar pool deck items...


# 3. Word embedding

## 3.1 CountVectorizer

In [2]:
cv = CountVectorizer(stop_words='english')
cv.fit(df.text_noun)
X = cv.transform(df.text_noun).transpose()
pd.DataFrame(X.toarray(), cv.get_feature_names()).head()

## 3.2 TF-IDF

In [3]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(df.text_noun)
X2 = tfidf.transform(df.text_noun).transpose()
pd.DataFrame(X2.toarray(), tfidf.get_feature_names()).head()

# 4. Topic modeling

In [ ]:
def topic_search(data, model, num_topics, passes):
    corpus = matutils.Sparse2Corpus(data)
    id2word = dict((v, k) for k, v in model.vocabulary_.items())
    lda = models.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word, passes=passes)
    lda_corpus = lda[corpus]  # Transform the docs from the word space to the topic space (like "transform" in sklearn)
    lda_docs = [doc for doc in lda_corpus]
    return lda.print_topics(), lda_docs

In [19]:
# Using CounterVectorizer
topics, topic_vectors = topic_search(X, cv, 5, 5) 

In [25]:
topics

[(0,
  '0.042*"room" + 0.036*"hotel" + 0.012*"rooms" + 0.011*"casino" + 0.009*"pool" + 0.007*"day" + 0.007*"stay" + 0.007*"buffet" + 0.007*"view" + 0.006*"desk"'),
 (1,
  '0.029*"burger" + 0.015*"fries" + 0.012*"club" + 0.009*"burgers" + 0.009*"line" + 0.008*"drinks" + 0.007*"restaurant" + 0.007*"bar" + 0.007*"order" + 0.007*"table"'),
 (2,
  '0.014*"buffet" + 0.011*"restaurant" + 0.010*"dinner" + 0.009*"steak" + 0.007*"meal" + 0.007*"chicken" + 0.007*"menu" + 0.007*"table" + 0.006*"experience" + 0.006*"order"'),
 (3,
  '0.046*"room" + 0.028*"hotel" + 0.016*"rooms" + 0.011*"casino" + 0.010*"pool" + 0.008*"floor" + 0.007*"day" + 0.007*"view" + 0.007*"bathroom" + 0.007*"staff"'),
 (4,
  '0.018*"pizza" + 0.013*"chocolate" + 0.011*"cirque" + 0.009*"line" + 0.007*"stage" + 0.006*"seats" + 0.006*"buffet" + 0.006*"way" + 0.006*"shows" + 0.005*"music"')]

In [ ]:
topics, topic_vectors = topic_search(X2, tfidf, 5, 5)

In [31]:
topics

([(0,
   '0.000*"room" + 0.000*"hotel" + 0.000*"acts" + 0.000*"acrobatics" + 0.000*"cirque" + 0.000*"shack" + 0.000*"tent" + 0.000*"gazillionaire" + 0.000*"audience" + 0.000*"zumanity"'),
  (1,
   '0.001*"room" + 0.001*"hotel" + 0.001*"restaurant" + 0.000*"experience" + 0.000*"dinner" + 0.000*"way" + 0.000*"buffet" + 0.000*"bar" + 0.000*"menu" + 0.000*"day"'),
  (2,
   '0.000*"burger" + 0.000*"room" + 0.000*"fries" + 0.000*"restaurant" + 0.000*"hotel" + 0.000*"burgers" + 0.000*"bar" + 0.000*"menu" + 0.000*"table" + 0.000*"meal"'),
  (3,
   '0.000*"cirque" + 0.000*"acrobatics" + 0.000*"stage" + 0.000*"performers" + 0.000*"soleil" + 0.000*"du" + 0.000*"stunts" + 0.000*"ka" + 0.000*"shows" + 0.000*"storyline"'),
  (4,
   '0.000*"macarons" + 0.000*"buffet" + 0.000*"room" + 0.000*"line" + 0.000*"bouchon" + 0.000*"steak" + 0.000*"hotel" + 0.000*"club" + 0.000*"restaurant" + 0.000*"chocolate"')],
 [[(0, 0.010511828559402941),
   (1, 0.95795251757019961),
   (2, 0.010512103410533899),
   (3, 0

# 5. Categorizing business based on topics

In [23]:
def matching(topic_vectors):
    topic_number = []
    for i in range(len(topic_vectors)):
        a=[]
        for j in range(len(topic_vectors[i])):
            a.append(topic_vectors[i][j][1])
        topic_number.append(topic_vectors[i][a.index(max(a))][0])
    df_topics=pd.DataFrame(pd.DataFrame({'business_id': df.business_id, 'topic_number': topic_number}))

    categories = [list(df_business.loc[df_business.business_id == item].categories) for item in df_topics.business_id]
    df_topics['categories'] = categories
    return df_topics

df_topics = matching(topic_vectors)

In [533]:
df_topics.loc[df_topics.topic_number == 0]

,business_id,topic_number,categories,business_name,star,review_count
6,5LNZ67Yw9RD6nf4_UhXOjw,0,"[[Casinos, Event Planning & Services, Hotels &...",The Cosmopolitan of Las Vegas,4.0,3621
7,5iSmZO0SrKU6EoXK_1M8Kw,0,"[[Casinos, Hotels & Travel, Event Planning & S...",Monte Carlo Hotel And Casino,2.5,2336
9,6Q7-wkCPc1KF75jZLOTcMw,0,"[[Arts & Entertainment, Hotels & Travel, Casin...",Circus Circus Las Vegas Hotel and Casino,2.5,1993
15,BLIJ-p5wYuAhw6Pp6mh6mw,0,"[[Hotels & Travel, Event Planning & Services, ...",Planet Hollywood Las Vegas Resort & Casino,3.0,2561
19,DfgZlNgKwBvCpA_0alumXw,0,"[[Arts & Entertainment, Casinos, Resorts, Hote...",The Mirage,3.5,1885
22,El4FC8jcawUVgw_0EIcbaQ,0,"[[Restaurants, Casinos, Hotels & Travel, Event...",MGM Grand Hotel,3.0,3285
41,NCFwm2-TDb-oBQ2medmYDg,0,"[[Street Art, Performing Arts, Public Services...",Fountains of Bellagio,4.5,1083
53,SMPbvZLSMMb7KU76YNYMGg,0,"[[Hotels, Casinos, Event Planning & Services, ...",ARIA Resort & Casino,3.5,3634
57,VyjyHoBg3KC5BSFRlD0ZPQ,0,"[[Hotels & Travel, Casinos, Hotels, Arts & Ent...",Caesars Palace Las Vegas Hotel & Casino,3.0,2260
59,WYw3Uf56DT5IwpaLNnCH5Q,0,"[[Resorts, Day Spas, Hotels, Event Planning & ...",Vdara Hotel,4.0,2235


# 6. Matching business data to review data

In [102]:
def merge(df, df_business):
    bn = [df_business.loc[df_business.business_id == df.business_id[i]].name.iloc[0] for i in range(len(df))]
    bc = [df_business.loc[df_business.business_id == df.business_id[i]].categories.iloc[0] for i in range(len(df))]
    ba = [df_business.loc[df_business.business_id == df.business_id[i]].address.iloc[0] for i in range(len(df))]
    bla = [df_business.loc[df_business.business_id == df.business_id[i]].latitude.iloc[0] for i in range(len(df))]
    blo = [df_business.loc[df_business.business_id == df.business_id[i]].longitude.iloc[0] for i in range(len(df))]

    df['business_name'] = bn
    df['business_categories'] = bc
    df['address'] = ba
    df['latitude'] = bla
    df['longitude'] = blo
    return df

df = merge(df, df_business)

In [104]:
df.head(50)

,business_id,cool,funny,stars,useful,text_clean,text_noun,business_name,business_categories,address,latitude,longitude
0,--9e1ONYQuAa-CB_Rrw7Tw,0.631389,0.471562,4.087113,0.942405,what can i say wowzers probably one of the be...,wowzers steak houses dinner sea tower wedge w...,Delmonico Steakhouse,"[Steakhouses, Cajun/Creole, Restaurants]",3355 Las Vegas Blvd S,36.123183,-115.169190
1,0AQnRQw34IQW9-1gJkYnMA,0.307479,0.323176,2.943675,0.598338,me and my best friend stayed at the monte car...,friend monte carlo diablo s hotel didn feel d...,Diablo's Cantina,"[Restaurants, Mexican, Bars, Nightlife]",3770 Las Vegas Blvd S,36.104133,-115.173544
2,0NmTwqYEQiKErDv4a55obg,0.801971,0.573477,4.120968,1.252688,stars goes to the truffled mushroom with pole...,stars mushroom stars basil spaghetti ambience ...,Scarpetta,"[Italian, Restaurants]",3708 Las Vegas Blvd S,36.110241,-115.174288
3,2weQS-RnoOBhb1KsHKyoSQ,0.767622,0.734573,3.637749,1.209140,over one hour wait for sunday brunch not a hu...,hour wait brunch selection breakfast items des...,The Buffet,"[Buffets, Restaurants]",3131 Las Vegas Blvd S,36.126887,-115.165590
4,4GXII-GU7S0ZyU6ElkhscQ,0.496835,0.787975,2.352848,1.131329,yeah this place is a friggin joke i live hear...,yeah joke i hear pool sulfar pool deck items...,Hooters Casino Hotel,"[Chicken Wings, Casinos, American (New), Arts ...",115 E Tropicana Ave,36.099786,-115.168155
5,4JNXUYY8wbaaDmk3BPzlWw,0.470479,0.343508,4.113070,0.670249,very chic although the menu items doesnt scre...,menu items cuisine s s i salmon platter i s ...,Mon Ami Gabi,"[French, Steakhouses, Breakfast & Brunch, Rest...",3655 Las Vegas Blvd S,36.112827,-115.172581
6,5LNZ67Yw9RD6nf4_UhXOjw,0.985643,0.886803,3.874379,1.695472,first stay here it is beautiful luxury is cor...,stay luxury room floor view size son sofa bed ...,The Cosmopolitan of Las Vegas,"[Casinos, Event Planning & Services, Hotels & ...",3708 Las Vegas Blvd S,36.109746,-115.173888
7,5iSmZO0SrKU6EoXK_1M8Kw,0.463383,0.495503,2.688651,1.002570,can i give zero stars had the worst time in l...,zero stars thanks monte carlo lobby i guess ...,Monte Carlo Hotel And Casino,"[Casinos, Hotels & Travel, Event Planning & Se...",3770 Las Vegas Blvd S,36.132005,-115.161450
8,6Ct57qgmXwOnzfSZoUGh0Q,0.540889,0.629749,2.924018,1.180940,my wife and i recently stayed at bally s and ...,wife i hotel upgrade wife i employees maid sta...,Bally's Las Vegas Hotel & Casino,"[Casinos, Event Planning & Services, Hotels & ...",3645 Las Vegas Blvd S,36.114321,-115.170866
9,6Q7-wkCPc1KF75jZLOTcMw,0.537845,0.935338,2.331328,1.211529,i always visit circus circus just for the car...,i circus carnival games arcades carnival midwa...,Circus Circus Las Vegas Hotel and Casino,"[Arts & Entertainment, Hotels & Travel, Casino...",2880 S Las Vegas Blvd,36.136304,-115.162227


# 7. Recommendation system

In [ ]:
df_wide = pd.pivot_table(df, values=["stars"], index=["business_name", "user_id"], aggfunc=np.mean).unstack()
df_wide = df_wide.fillna(0) # replace NaN with 0
dists = cosine_similarity(df_wide)
dists = pd.DataFrame(dists, columns=df_wide.index)
dists.index = dists.columns

In [ ]:
# Recommend based on the similarity of inputs and outputs
def get_similar(businesses, n=None):

    businesses = [business for business in businesses if business in dists.columns]
    busi_summed = dists[businesses].apply(lambda row: np.sum(row), axis=1).sort_values(ascending=False)
    ranked_busi = busi_summed.index[busi_summed.index.isin(businesses)==False]
    ranked_business = ranked_busi.tolist()
    if n is None:
        return ranked_business
    else:
        return ranked_business[:n]

In [ ]:
for i, business in enumerate(get_similar(["Bacchanal Buffet"], 10)):
    print("%d) %s" % (i+1, business))